This Jupyter notebook is centered on calculating and storing descriptors for time series data using the `D2C` class from the `d2c.descriptors` module. It works with previously generated data stored in a directory and processes each file to extract time series descriptors. The notebook employs the Python libraries pandas and tqdm to manage and monitor data operations.

### Breakdown of the Notebook:

1. **Setup and Imports:**
   - Essential libraries like `pickle`, `os`, `pandas`, and `tqdm` are imported. Additionally, `D2C` and `DataLoader` from the `d2c.descriptors` module are used for processing the data.
   - Constants such as `N_JOBS` (number of parallel jobs), `SEED`, `MB_SIZE` (Markov blanket size), `COUPLES_TO_CONSIDER_PER_DAG`, and `maxlags` are defined for the descriptor calculations.

2. **Processing Each Data File:**
   - The notebook iterates over each file in the sorted order from the data directory. For each file, it extracts metadata like process number, number of variables, neighborhood size, and noise standard deviation from the filename.
   - A `DataLoader` instance is created and used to load the data from the respective file. The loaded data includes observations and DAGs (directed acyclic graphs) which represent the time series data and its underlying relationships.

3. **Descriptor Calculation:**
   - The `D2C` (Data to Causality) object is initialized with the loaded data and the predefined settings (like seed, number of jobs, etc.). The initialization prepares the framework for descriptor calculations.
   - Descriptors are computed for the data, which involves statistical and causal analysis to derive meaningful metrics that describe the interactions and dependencies within the data.

4. **Saving the Descriptors:**
   - The resulting dataframe containing descriptors is augmented with metadata columns indicating the process ID, number of variables, neighborhood size, and noise standard deviation.
   - The dataframe is then saved as a pickle file in a designated directory for descriptors. Each file is named systematically to reflect the configuration of the data it describes.

### Key Features:
- **Efficient Data Handling:** The notebook effectively handles large datasets by iterating through files and processing them individually, making efficient use of computational resources.
- **Progress Monitoring:** Integration of `tqdm` provides a progress bar for tracking the completion of processing across multiple files.
- **Detailed Metadata Management:** Metadata from filenames is used to annotate the descriptor results, ensuring that each output file is traceable back to its input conditions.

This notebook automates the computation of detailed descriptors for time series datasets, providing a scalable solution for analyzing large volumes of data in a structured and reproducible manner.

This notebook generates the training descriptors. 
For each file; only a subset of 300 couples of variables are chosen, according to the following logic

- 100 causal couples: A selection of 20 pairs of variables that are causally related according to the DAG. These serve as positive examples.
- 100 opposite couples: For each of the causal couples selected, the corresponding opposite pair (effect as cause and cause as effect) is also chosen. These pairs, despite being theoretically informative, do not respect the temporal ordering and thus they won't appear in the results returned by competitor methods. For this reason, they will be excluded from the evaluation phase later on, although they will remain in the training set of our classifier.
- 100 additional noncausal couples: To compensate for the exclusion of the opposite couples in validation and ensure a balanced dataset. Unlike the opposite ones, these pairs are chosen based on their lack of causal connection in the DAG and are more likely to resemble noncausal relationships encountered in real-world scenarios.

In [ ]:
import pickle 
import os
import pandas as pd
from d2c.descriptors_generation import D2C, DataLoader
from tqdm import tqdm

In [ ]:
N_JOBS = 40 # number of jobs to run in parallel. For D2C, parallelism is implemented at the observation level: each observation from a single file is processed in parallel
SEED = 42 # random seed for reproducibility
MB_SIZE = 2 # size to consider when estimating the markov blanket. This is only useful if the MB is actually estimated
COUPLES_TO_CONSIDER_PER_DAG = -1 # edges that are considered in total to compute descriptors, for each TS. This can speed up the process. If set to -1, all possible edges are considered
maxlags = 5 # maximum lags to consider when considering variable couples

For which files do we want to compute our descriptors? We choose the ones with fixed `noise_std == 0.01`and fixed `max neighbordhood size==2`

In [ ]:
"""
This script takes the files in the input folder and filters them according to the parameters of the process.
"""

input_folder = '/home/jpalombarini/td2c/notebooks/paper_td2c/.data/'

to_process = [] # list of files to process

# This loop is used to filter the files to process and obtain the parameters of the process
# The resulting list will be used to parallelize the process and will be passed to the DataLoader.
# The result is of the form (file, gen_process_number, n_variables, max_neighborhood_size, noise_std)
# asnd is saved in the to_process list.
for file in sorted(os.listdir(input_folder)): 
    gen_process_number = int(file.split('_')[0][1:])
    n_variables = int(file.split('_')[1][1:])
    max_neighborhood_size = int(file.split('_')[2][2:])
    noise_std = float(file.split('_')[3][1:-4])

    if noise_std != 0.01:
        continue
    
    if max_neighborhood_size != 2:
        continue

    # if n_variables != 5:
    #     continue

    to_process.append(file)

Now we use the D2C method to compute the descriptors. <br>
We use `mb_estimator='ts'` to select the past and future values of the considered variable as its Markov Blanket <br>
We use `cmi='original'` to estimate conditional mutual information as in (Bontempi et al, 2015) <br>


In [ ]:
# ETA 2h with 40 jobs!!

"""
This sccript processes the files in the input folder and saves the descriptors in the output folder.
"""

output_folder = '/home/jpalombarini/td2c/notebooks/paper_td2c/.descriptors_ts_cmiknn_entropy/'  

# create output folder if it does not exist
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

# This loop processes the files in the input folder (to_process) and saves the descriptors in the output folder.

# At first, we collect the parameters of the process from the file name.
for file in tqdm(to_process):
    gen_process_number = int(file.split('_')[0][1:])
    n_variables = int(file.split('_')[1][1:])
    max_neighborhood_size = int(file.split('_')[2][2:])
    noise_std = float(file.split('_')[3][1:-4])

# The DataLoader is initialized with the parameters of the process. 
    dataloader = DataLoader(n_variables = n_variables,
                    maxlags = maxlags)
    dataloader.from_pickle(input_folder+file)

# The D2C object is initialized with the DataLoader and the parameters of the process.
    d2c = D2C(observations=dataloader.get_observations(), 
            dags=dataloader.get_dags(), 
            couples_to_consider_per_dag=COUPLES_TO_CONSIDER_PER_DAG, 
            MB_size=MB_SIZE, 
            n_variables=n_variables, 
            maxlags=maxlags,
            seed=SEED,
            n_jobs=N_JOBS,
            full=True,
            quantiles=True,
            normalize=True,
            cmi='original',
            mb_estimator='ts')

    d2c.initialize() # initializes the D2C object

    descriptors_df = d2c.get_descriptors_df()  # computes the descriptors

    descriptors_df.insert(0, 'process_id', gen_process_number)
    descriptors_df.insert(2, 'n_variables', n_variables)
    descriptors_df.insert(3, 'max_neighborhood_size', max_neighborhood_size)
    descriptors_df.insert(4, 'noise_std', noise_std)

    # The descriptors are saved in the output folder as a pickle file.
    descriptors_df.to_pickle(output_folder+f'P{gen_process_number}_N{n_variables}_Nj{max_neighborhood_size}_n{noise_std}_MB{MB_SIZE}.pkl')

Now we use `mb_estimator='ts'` to select the past and future values of the considered variable as its Markov Blanket <br>
We use `cmi='cmiknn'` to estimate conditional mutual information with knncmi <br>

In [ ]:
# for this step it's required to clone the knncmi repository and install it
#
# git clone https://github.com/omesner/knncmi.git
# cd knncmi
# pip install .

In [ ]:
to_process

In [ ]:
"""
This script removes the files that were already processed from the to_process list and creates the output folder for knnCMI descriptors.
"""

output_folder = '/home/jpalombarini/td2c/notebooks/paper_td2c/.descriptors_ts_cmiknn_entropy/'  

#create output folder if it does not exist
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

# remove already processed files
to_process_copy = to_process.copy()
for file in os.listdir(output_folder):
    name = file[:-8]+file[-4:]
    if name in to_process_copy:
        to_process.remove(name)

In [ ]:
# This loop processes the files in the input folder (to_process) and saves the descriptors in the output folder.

# At first, we collect the parameters of the process from the file name.
for file in tqdm(to_process):
    gen_process_number = int(file.split('_')[0][1:])
    n_variables = int(file.split('_')[1][1:])
    max_neighborhood_size = int(file.split('_')[2][2:])
    noise_std = float(file.split('_')[3][1:-4])

# The DataLoader is initialized with the parameters of the process.
    dataloader = DataLoader(n_variables = n_variables,
                    maxlags = maxlags)
    dataloader.from_pickle(input_folder+file)

# The D2C object is initialized with the DataLoader and the parameters of the process.
    d2c = D2C(observations=dataloader.get_observations(), 
            dags=dataloader.get_dags(), 
            couples_to_consider_per_dag=COUPLES_TO_CONSIDER_PER_DAG, 
            MB_size=MB_SIZE, 
            n_variables=n_variables, 
            maxlags=maxlags,
            seed=SEED,
            n_jobs=N_JOBS,
            full=True,
            quantiles=True,
            normalize=True,
            cmi='cmiknn_3',
            mb_estimator='ts')

    d2c.initialize() # initializes the D2C object

    descriptors_df = d2c.get_descriptors_df() # computes the descriptors

    descriptors_df.insert(0, 'process_id', gen_process_number)
    descriptors_df.insert(2, 'n_variables', n_variables)
    descriptors_df.insert(3, 'max_neighborhood_size', max_neighborhood_size)
    descriptors_df.insert(4, 'noise_std', noise_std)

    # The descriptors are saved in the output folder as a pickle file.
    descriptors_df.to_pickle(output_folder+f'P{gen_process_number}_N{n_variables}_Nj{max_neighborhood_size}_n{noise_std}_MB{MB_SIZE}.pkl')

In [ ]:
descriptors = {}
# for output_folder in ['/home/jpalombarini/td2c/notebooks/paper_td2c/descriptors_original_full/',
#                      '/home/jpalombarini/td2c/notebooks/paper_td2c/descriptors_cmiknn_full/',
#                      '/home/jpalombarini/td2c/notebooks/paper_td2c/descriptors_cmiknn_full_k5/',
#                      '/home/jpalombarini/td2c/notebooks/paper_td2c/descriptors_cmiknn_full_k15/']:
for output_folder in ['/home/jpalombarini/td2c/notebooks/paper_td2c/.descriptors_ts_cmiknn_entropy/',
                    '/home/jpalombarini/td2c/notebooks/paper_td2c/.descriptors_ts_original_entropy/']:
    for single_output_folder in [output_folder]:
        dfs = []
        for file in os.listdir(single_output_folder):
            df = pd.read_pickle(single_output_folder+file)
            dfs.append(df)
        descriptors[single_output_folder] = pd.concat(dfs)

In [ ]:
# It returns the columns' names of the descriptors
descriptors[single_output_folder].columns

In [ ]:
"""
This script performs the Leave-One-Process-Out Cross-Validation (LOPO-CV) for the descriptors using a Random Forest classifier.
The scope is to evaluate the performance of the descriptors in predicting causal relationships.
The performance is evaluated using the ROC-AUC metric.
"""

from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score


# This function performs the Leave-One-Process-Out Cross-Validation (LOPO-CV) for the descriptors using a Random Forest classifier.
def perform_lopo_cv(descriptors_df):
    roc_aucs = {} # dictionary to store the ROC-AUC values for each process

    # This loop performs the LOPO-CV
    for process_id in descriptors_df['process_id'].unique(): 
        df_test = descriptors_df[descriptors_df['process_id'] == process_id]
        df_train = descriptors_df[descriptors_df['process_id'] != process_id]

        X_train = df_train.drop(columns=['process_id','graph_id','n_variables','max_neighborhood_size','noise_std','edge_source','edge_dest','is_causal'])
        y_train = df_train['is_causal']

        X_test = df_test.drop(columns=['process_id','graph_id','n_variables','max_neighborhood_size','noise_std','edge_source','edge_dest','is_causal'])
        y_test = df_test['is_causal']

        clf = RandomForestClassifier(n_estimators=50, random_state=42, n_jobs=50, max_depth=None)
        clf.fit(X_train, y_train)
        y_pred = clf.predict(X_test)
        y_pred_proba = clf.predict_proba(X_test)[:,1]

        roc_auc = roc_auc_score(y_test, y_pred_proba)
        roc_aucs[process_id] = roc_auc
    return roc_aucs


# rocs = {}
# for key in descriptors.keys():
#     rocs[key] = perform_lopo_cv(descriptors[key])

In [ ]:
#original mi, ts mb, less nj, a lot of couples 
perform_lopo_cv(descriptors[single_output_folder])


In [ ]:
#original mi, original mb, less nj
perform_lopo_cv(descriptors[single_output_folder])


In [ ]:
#original mi, original mb
perform_lopo_cv(descriptors[single_output_folder])


In [ ]:
#cmiknn, original mb
perform_lopo_cv(descriptors[single_output_folder])


In [ ]:
#cmiknn, tsmb
perform_lopo_cv(descriptors[single_output_folder])


In [ ]:
perform_lopo_cv(descriptors['/home/jpalombarini/td2c/notebooks/paper_td2c/.descriptors_ts_original_entropy/'])

In [ ]:
perform_lopo_cv(descriptors['/home/jpalombarini/td2c/notebooks/paper_td2c/.descriptors_ts_cmiknn_entropy/'])

In [ ]:
{13: 0.9317180555555555,
 3: 0.8845465277777778,
 16: 0.9710965277777778,
 11: 0.9660762152777778,
 19: 0.954526388888889,
 14: 0.9804326388888889,
 7: 0.9450670138888889,
 4: 0.9019128472222222,
 9: 1.0,
 20: 0.9452003472222222,
 10: 0.9563795138888889,
 15: 1.0,
 2: 0.9165769097222222,
 8: 0.9330982638888888,
 12: 1.0,
 18: 1.0,
 6: 0.928396875,
 1: 0.9210937499999999}

In [ ]:
# plot results
import matplotlib.pyplot as plt
import seaborn as sns

rocs = {}
for key in descriptors.keys():
    rocs[key] = perform_lopo_cv(descriptors[key])

rocs_df = pd.DataFrame(rocs)

plt.figure(figsize=(10,5))
sns.boxplot(data=rocs_df)
plt.ylabel('ROC AUC')
plt.xticks([0,1,2],['Original','Original-MBTS', 'CMIKNN-3-MBTS'])
plt.show()



In [ ]:
#barplot pairwise

rocs_df = pd.DataFrame(rocs)
rocs_df = rocs_df.T
rocs_df = rocs_df.reset_index()
rocs_df = rocs_df.melt(id_vars='index')
rocs_df.columns = ['process_id','method','roc_auc']

plt.figure(figsize=(10,5))
sns.barplot(data=rocs_df, x='process_id', y='roc_auc', hue='method')
plt.ylabel('ROC AUC')
plt.show()

# barplot same process_id side by side
rocs_df = pd.DataFrame(rocs)
rocs_df = rocs_df.T
rocs_df = rocs_df.reset_index()
rocs_df = rocs_df.melt(id_vars='index')
rocs_df.columns = ['process_id','method','roc_auc']

plt.figure(figsize=(10,5))
sns.barplot(data=rocs_df, x='method', y='roc_auc', hue='process_id')
plt.ylabel('ROC AUC')
plt.show()


In [ ]:
# Some minor feature importance
from imblearn.ensemble import BalancedRandomForestClassifier
df = descriptors['/home/jpalombarini/td2c/notebooks/paper_td2c/.descriptors_ts_cmiknn_entropy/']

df_train = df[df['process_id'] >= 10]
df_test = df[df['process_id'] < 10]

X_train = df_train.drop(columns=['process_id', 'graph_id', 'n_variables','max_neighborhood_size','noise_std','edge_source','edge_dest','is_causal'])
y_train = df_train['is_causal']

X_test = df_test.drop(columns=['process_id', 'graph_id' ,'n_variables','max_neighborhood_size','noise_std','edge_source','edge_dest','is_causal'])
y_test = df_test['is_causal']

clf = BalancedRandomForestClassifier(n_estimators=50, random_state=42, n_jobs=50, sampling_strategy='all', replacement=True)
# clf = RandomForestClassifier(n_estimators=50, random_state=42, n_jobs=50)
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)
y_pred_proba = clf.predict_proba(X_test)[:,1]

roc_auc = roc_auc_score(y_test, y_pred_proba)
print(roc_auc)

In [ ]:
import numpy as np
# Some minor feature importance
df = descriptors['/home/jpalombarini/td2c/notebooks/paper_td2c/.descriptors_ts_cmiknn_entropy/']

X = df.drop(columns=['process_id', 'graph_id', 'n_variables','max_neighborhood_size','noise_std','edge_source','edge_dest','is_causal'])
y = df['is_causal']


In [ ]:

clf = RandomForestClassifier(n_estimators=50, random_state=42, n_jobs=50, max_depth=None)
clf.fit(X, y)

importances = clf.feature_importances_
indices = np.argsort(importances)[::-1]

# Print the feature ranking
print("Feature ranking:")
for f in range(X.shape[1]):
    print(f"{f+1}. feature {X.columns[indices[f]]} ({importances[indices[f]]})")



In [ ]:
df_train = df[df['process_id'] >= 10]
df_test = df[df['process_id'] < 10]

X_train = df_train.drop(columns=['process_id', 'graph_id', 'n_variables','max_neighborhood_size','noise_std','edge_source','edge_dest','is_causal'])
y_train = df_train['is_causal']


X_test = df_test.drop(columns=['process_id', 'graph_id' ,'n_variables','max_neighborhood_size','noise_std','edge_source','edge_dest','is_causal'])
y_test = df_test['is_causal']


# clf = BalancedRandomForestClassifier(n_estimators=50, random_state=42, n_jobs=50, sampling_strategy='all', replacement=True)
# # clf = RandomForestClassifier(n_estimators=50, random_state=42, n_jobs=50)
# clf.fit(X_train, y_train)

from sklearn.neural_network import MLPClassifier
clf = MLPClassifier(random_state=42, hidden_layer_sizes=(100, 100), max_iter=1000)
clf.fit(X_train, y_train)

y_pred = clf.predict(X_test)
y_pred_proba = clf.predict_proba(X_test)[:,1]

roc_auc = roc_auc_score(y_test, y_pred_proba)
print(roc_auc)

In [ ]:
#accuracy and other metrics
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)

print(f'Accuracy: {accuracy}')
print(f'Precision: {precision}')
print(f'Recall: {recall}')
print(f'F1: {f1}')


In [ ]:
topick